In [210]:
## De functie die tf-records leest en ze uiteindelijk weer wegschrijft (maar dan met minder labels) heet readTfRecord
## Deze functie roept andere functies aan: getLabels en getLabelIndices

import numpy as np
import pandas as pd
import re
import os
import random
from collections import Counter

import tensorflow as tf
from tensorflow import gfile

In [211]:
train_data_pattern = 'tfrecords/train_data_old/*.tfrecord'
train_data_folder_new = 'tfrecords/train_data_new'
test_data_pattern = 'tfrecords/test_data_old/*.tfrecord'
test_data_folder_new = 'tfrecords/test_data_new'

In [236]:
## Keep list of all new labels (to see which ones occur often)
all_labels = []
label_combinations = []
# Maximal proportion for labels
max_prop = 0.005

In [237]:
## Open old csv file and alternated csv file with the super classes
mid_to_label_old = pd.read_csv('csv_files/class_labels_indices.csv')
mid_to_label_new = pd.read_csv('csv_files/class_labels_indices_proportions.csv', sep=";")

In [253]:
## Deze functie checkt of een example wordt gemaakt (met kans)
def checkIfNewExample(labels):
    
    ## Veel combi's komen erg vaak voor. Ik heb handmatig kansen verzonnen waarop een example wordt gemaakt
    ## In de toekomst moet dit uiteraard beter
    if len(labels)>1:
        rand = random.random()
        if set(labels)==set([0,4]):
            if rand <= 0.015:
                return True
            else:
                return False
        elif set(labels)==set([4,5]):
            if rand <= 0.03:
                return True
            else:
                return False
        elif set(labels)==set([0,6]) or set(labels)==set([6,8,17]) or set(labels)==set([0,4,5]) or set(labels)==set():
            if rand <= 0.12:
                return True
            else:
                return False
        elif set(labels)==set([0,4,5]) or  set(labels)==set([6,7]) or set(labels)==set([0,2,3]) or set(labels)==set([0,6,8,17]):
            if rand <= 0.25:
                return True
            else:
                return False
        elif set(labels)==set([6,10]) or set(labels)==set([0,5]) or set(labels)==set([6,8,25]) or set(labels)==set([6,9]) or set(labels)==set([0,6,7]) or set(labels)==set([6,8,17,27,33]):
            if rand <= 0.4:
                return True
            else:
                return False
        elif set(labels)==set([6,27,33]) or set(labels)==set([6,8,17,18]) or set(labels)==set([6,8,17,19]) or set(labels)==set([2,3]) or set(labels)==set([0,6,8,20]):
            if rand <= 0.7:
                return True
            else:
                return False
        else:
            return True
    else:
        label = list(labels)[0]
        proportion = mid_to_label_new.loc[np.int(label), "proportion"]
        rand = random.random()
        ## Als proportie niet max_prop overschrijdt, return true
        if label==33 or label==34:
            if rand <= 0.7:
                return True
            else:
                return False
        elif proportion < max_prop:
            return True
        else:
            prob = max_prop/proportion
            if rand <= prob:
                return True
            else:
                return False
    return False

In [239]:
#### Find new label for each of old label (527 -> 31)

def getNewLabel(label_old):
    ## Get m-id that matches with index
    mid = mid_to_label_old.loc[mid_to_label_old['index']==label_old, 'mid']

    ## Find the new label belonging to this m-id
    label_new = mid_to_label_new.loc[mid_to_label_new['mid']==str(mid.values[0]), 'index']
    
    return label_new

In [240]:
def createExample(example_old, new_labels):
    
    #print("New labels: " + str(new_labels))
    
    audio_embedding = example_old.feature_lists.feature_list['audio_embedding']
    feature_lists = tf.train.FeatureLists(feature_list={"audio_embedding": audio_embedding})

    label_feat = tf.train.Feature(int64_list=tf.train.Int64List(value=new_labels))
    video_id = example_old.context.feature['video_id'].bytes_list.value
    videoid_feat = tf.train.Feature(bytes_list=tf.train.BytesList(value=video_id)) 

    context_feats = tf.train.Features(feature={"labels": label_feat, "video_id": videoid_feat})

    sequence_example = tf.train.SequenceExample(context=context_feats, feature_lists=feature_lists)
    
    return sequence_example

In [241]:
def getParents(label, parents):
    
    parent = int(mid_to_label_new.loc[mid_to_label_new['index']==label, 'parent'])
    
    if parent==999:
        return parents
    else:
        parents.append(parent)
        return getParents(parent, parents)
    return parent

def getLabelsWithParents(labels):
    total_labels = labels
    for label in labels:
        parents = getParents(label, [])
        total_labels.extend(parents)
    return(set(total_labels))

In [242]:
#### Lees tf-records en iterate over de examples#### Lee 
#### Voor elk example wordt de video-id gepakt en de bijbehorende lijst m-id's gezocht (in balanced_segment.csv)

## One tf-record has several examples
## This function checks for each example if it has a label we are interested in
## If so, we compute the one-hot vector
## Input: filename of tf-record
## Result: write new tf-record

def readWriteTfRecord(tfrecords_filename, new_folder):
    
    ## Create writer that writes new tf-record in train_data_folder_new
    path_tfrecord_new = str(new_folder + '/' + tfrecords_filename.split('\\')[-1])
    writer = tf.python_io.TFRecordWriter(path_tfrecord_new)
    nr_new_examples = 0
    
    ## Iterator that goes through all examples of tf-record
    record_iterator = tf.python_io.tf_record_iterator(path=tfrecords_filename)

    
    for string_record in record_iterator:
        
        ## Take one example
        example = tf.train.SequenceExample()
        example.ParseFromString(string_record)
 
        ## Get labels and find for each label the m-code in class_label_indices_old
        labels = example.context.feature['labels'].int64_list.value
        
        new_labels = []
        for label in labels:
            ## Hij returnt een set van cellen (of leeg of van 1 rij)
            label_new = getNewLabel(label)
            
            if(label_new.shape[0]>0):
                new_labels.append(label_new.values[0])
        
        ## Als enkele (grand)parents ontbreken, voeg ze toe
        new_labels = getLabelsWithParents(new_labels)
        ## If any of the labels left (for our model), write new tf-record
        if len(new_labels) > 0 and checkIfNewExample(new_labels):
        #if len(new_labels) > 0:
            ## Add to all labels
            all_labels.extend(new_labels)
            label_combinations.append(new_labels)
   
            sequence_example = createExample(example, new_labels)
            nr_new_examples += 1
            
            ## Write the example
            writer.write(sequence_example.SerializeToString())
            #print('Example gemaakt! Video-id: ' + str(example.context.feature['video_id'].bytes_list.value))

    #print("nr new examples: " + str(nr_new_examples))
    # Only write if any examples created

    if nr_new_examples > 0:
        writer.close()
        #print("tfrecord written")
    else:
        #print("empty file.. remove: " + path_tfrecord_new)
        os.remove(path_tfrecord_new)

In [243]:
## Read and rewrite all train files
#train_data_pattern = 'tfrecords/train_data_old/*2.tfrecord'
files = gfile.Glob(train_data_pattern)

for file in files:
    
    ## Read a file and rewrite it
    readWriteTfRecord(file, train_data_folder_new)

In [244]:
sum_occur = 0
cnt_labels = Counter(all_labels)
for i in cnt_labels.most_common(100):
    sum_occur = sum_occur + i[1]
    print(mid_to_label_new.loc[np.int(i[0])]['display_name'], ' : ', str(i[1]))

Vehicle  :  28757
Speech  :  23129
Motor vehicle (road)  :  16886
Music  :  11805
Animal  :  10908
Car  :  9601
Engine  :  8741
Domestic animals, pets  :  5080
Accelerating, revving, vroom  :  3760
Tools  :  3330
Emergency vehicle  :  2882
Medium engine (mid frequency)  :  2754
Idling  :  2718
Boat, Water vehicle  :  2233
Motorcycle  :  2075
Bus  :  1852
Musical instrument  :  1849
Heavy engine (low frequency)  :  1607
Gunshot, gunfire  :  1606
Rail transport  :  1587
Car passing by  :  1564
Aircraft  :  1538
Vehicle horn, car horn, honking  :  1322
Police car (siren)  :  1202
Power tool  :  1154
Fire engine, fire truck (siren)  :  1096
Conversation  :  861
Ambulance (siren)  :  641
Machine gun  :  579
Chainsaw  :  576
Traffic noise, roadway noise  :  568
Engine starting  :  443
Hammer  :  135
Train wheels squealing  :  128
Jackhammer  :  60


In [245]:
## Tellen van lijsten in lijsten is lastig, vandaar dat ik er strings van maak: [0,4] wordt ["0_4"]
str_combis = []

for combi in label_combinations:
    str_combi = [str(i) for i in combi]
    str_combi = "_".join(str_combi)
    str_combis.append(str_combi)

In [249]:
## Geef meest voorkomende combinaties

cnt_label_combinations = Counter(str_combis)
for label_combination in cnt_label_combinations.most_common(50):
    label_names = []
    label_list = label_combination[0].split("_")
    for i in label_list:
        label_names.append(str(mid_to_label_new.loc[np.int(i)]['display_name']))
    label_names = ', '.join(label_names)
    print(label_names,': ', label_combination[1])

Animal, Domestic animals, pets :  3723
Speech, Animal :  3695
Speech :  3131
Music :  2935
Speech, Vehicle :  1257
Motor vehicle (road), Car, Vehicle :  1239
Animal :  1220
Speech, Car, Vehicle, Motor vehicle (road) :  1114
Vehicle, Boat, Water vehicle :  1112
Speech, Music :  1074
Motor vehicle (road), Motorcycle, Vehicle :  1024
Speech, Music, Musical instrument :  1015
Rail transport, Vehicle :  1012
Tools :  939
Motor vehicle (road), Bus, Vehicle :  908
Aircraft, Vehicle :  906
Tools, Power tool :  905
Vehicle :  884
Gunshot, gunfire :  851
Motor vehicle (road), Car, Car passing by, Vehicle :  851
Music, Musical instrument :  819
Motor vehicle (road), Car, Vehicle horn, car horn, honking, Vehicle :  780
Speech, Tools :  771
Speech, Conversation :  764
Accelerating, revving, vroom, Engine, Vehicle :  747
Music, Vehicle :  746
Speech, Music, Vehicle :  695
Speech, Vehicle, Boat, Water vehicle :  693
Speech, Motor vehicle (road), Bus, Vehicle :  686
Accelerating, revving, vroom, Vehic

In [247]:
sum_occur

155027

In [248]:
## Read and rewrite all test files
files = gfile.Glob(test_data_pattern)

for file in files:
    
    ## Read a file and rewrite it
    readWriteTfRecord(file, test_data_folder_new)

In [191]:
## Code die kolom toevoegt aan label-csv (proportion) en deze in de forloop vult

mid_to_label_new["proportion"] = 0

for i in range(mid_to_label_new.shape[0]):
    nr_occur = cnt_labels[i]
    mid_to_label_new.loc[i,"proportion"] = round(nr_occur/sum_occur,8)

In [192]:
## Schrijf bestandje weg
mid_to_label_new.to_csv("csv_files/class_labels_indices_proportions.csv", sep = ";")

0.010413249819069784